In [29]:
import pandas as pd
import requests
import os
import datetime

In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [238]:
av_api_key = os.environ['ALPHA_VANTAGE_API_KEY']

In [34]:
num_requests = 0

In [261]:
def getData(stock_symbol: str) -> object:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&outputsize=full&apikey={av_api_key}'
    r = requests.get(url)
    return r.json()

In [283]:
num_requests += 1
stock_data = getData('AMD')['Time Series (Daily)']

In [290]:
def simulate(day: datetime.datetime, data: object, return_threshold: float, buy_threshold: float, chunks: int, initial_amt: float) -> [float, list, float]:
    # [price, num_units]
    holdings = []
    cash = initial_amt
    num_units = chunks
    last_price = -1
    while(datetime.datetime.today() >= day):
        day_str = datetime.datetime.strftime(day, '%Y-%m-%d')
        if(day_str in data):
            last_price = float(data[day_str]['4. close'])
            data[day_str]['1. open'], data[day_str]['2. high'], data[day_str]['3. low'] = float(data[day_str]['1. open']),\
                                            float(data[day_str]['2. high']), float(data[day_str]['3. low'])

            #sell strategy
            extra_cash = 0
            extra_units = 0
            pop_list = []
            for i, stock_obj in enumerate(holdings):
                price, units_of_stock = stock_obj[0], stock_obj[1]
                pct_gain = (data[day_str]['2. high'] - price) / price
                if pct_gain > return_threshold:
                    pop_list.append(i)
                    revenue = (1 + return_threshold) * price * units_of_stock
                    profit = ((revenue - (price * units_of_stock)) * 0.63)
                    extra_cash += ((price * units_of_stock) + profit)
                    extra_units += 1
            counter = 0
            for i in pop_list:
                holdings.pop(i - counter)
                counter += 1
                
            #buy strategy
            pct_loss = (data[day_str]['3. low'] - data[day_str]['1. open']) / data[day_str]['1. open']
            if (-1 * pct_loss) > buy_threshold and cash > 0:
                holdings.append([data[day_str]['1. open'] * (1 - buy_threshold),
                                (cash / num_units) / (data[day_str]['1. open'] * (1 - buy_threshold))])
                cash -= (cash / num_units)
                num_units -= 1

            cash += extra_cash
            num_units += extra_units
                    
        day += datetime.timedelta(days=1)
        
    cur_assets = cash
    for stock_obj in holdings:
        price, num_units = stock_obj[0], stock_obj[1]
        cur_assets += (last_price * num_units)
    return [cash, holdings, ((cur_assets - initial_amt) / initial_amt)]

In [291]:
def baseline(day: datetime.datetime, data: object, initial_amt: float) -> [float, list, float]:
    # [price, num_units]
    holdings = []
    cash = initial_amt

    years_diff = datetime.datetime.today().year - day.year
    months_diff = datetime.datetime.today().month - day.month
    chunks = (years_diff * 12) + months_diff - 1

    prev_month = -1
    while(datetime.datetime.today() >= day):
        day_str = datetime.datetime.strftime(day, '%Y-%m-%d')
        if(day_str in data and cash > 0):
            last_price = float(data[day_str]['4. close'])
            #buy strategy
            data[day_str]['1. open'], data[day_str]['2. high'], data[day_str]['3. low'] = float(data[day_str]['1. open']),\
                                            float(data[day_str]['2. high']), float(data[day_str]['3. low'])
            if day.month != prev_month:
                holdings.append([data[day_str]['1. open'], (initial_amt / chunks) / data[day_str]['1. open']])
                prev_month = day.month
                cash -= (initial_amt / chunks)
        day += datetime.timedelta(days=1)

    cur_assets = cash
    for stock_obj in holdings:
        price, num_units = stock_obj[0], stock_obj[1]
        cur_assets += (last_price * num_units)
    return [cash, holdings, ((cur_assets - initial_amt) / initial_amt)]

In [292]:
start_date = '2015-01-01'
day = datetime.datetime.strptime(start_date, '%Y-%m-%d')
simulate(day, stock_data, 0.03, 0.03, 5, 10000)

[0.0,
 [[207.0077, 36.81561955098781],
  [194.5044, 39.18223303598798],
  [192.76809999999998, 39.78422655360076],
  [178.91649999999998, 47.56997922241803],
  [164.66719999999998, 52.6632693802555]],
 1.7042958880177477]

In [293]:
baseline(day, stock_data, 10000)[2]

12.380903055137637

In [294]:
thresholds = [i / 100 for i in range(1, 5)]
chunks = [i for i in range(1, 21)]
results = {}
# [chunk, return theshold, buy threshold, cash, holdings, utd return]
res_list = []

for chunk in chunks:
    for return_thresh in thresholds:
        for buy_thresh in thresholds:
            results[(chunk, return_thresh, buy_thresh)] = simulate(day, stock_data, return_thresh, buy_thresh, chunk, 10000)
            res_list.append([chunk, return_thresh, buy_thresh] + simulate(day, stock_data, return_thresh, buy_thresh, chunk, 10000))

In [295]:
print(f'BASELINE RETURN: {int(baseline(day, stock_data, 10000)[2] * 100)}%')
print('\n')
res_list.sort(reverse=True, key = lambda i: i[5])
top_ret = res_list
for i in range(0, 10):
    print(f"chunks: {top_ret[i][0]}")
    print(f'return threshold: {top_ret[i][1] * 100}%')
    print(f'buy threshold: {top_ret[i][2] * 100}%')
    print(f'TOTAL RETURN: {int(top_ret[i][5] * 100)}%')
    print('\n')

BASELINE RETURN: 1238%


chunks: 2
return threshold: 3.0%
buy threshold: 4.0%
TOTAL RETURN: 370%


chunks: 2
return threshold: 4.0%
buy threshold: 4.0%
TOTAL RETURN: 360%


chunks: 3
return threshold: 4.0%
buy threshold: 4.0%
TOTAL RETURN: 335%


chunks: 4
return threshold: 4.0%
buy threshold: 4.0%
TOTAL RETURN: 300%


chunks: 1
return threshold: 3.0%
buy threshold: 4.0%
TOTAL RETURN: 298%


chunks: 1
return threshold: 4.0%
buy threshold: 4.0%
TOTAL RETURN: 287%


chunks: 3
return threshold: 3.0%
buy threshold: 4.0%
TOTAL RETURN: 280%


chunks: 1
return threshold: 4.0%
buy threshold: 1.0%
TOTAL RETURN: 273%


chunks: 2
return threshold: 4.0%
buy threshold: 1.0%
TOTAL RETURN: 268%


chunks: 5
return threshold: 4.0%
buy threshold: 4.0%
TOTAL RETURN: 263%


